In [18]:
P = Primes()
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m,x):
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m,x):
    f = J(n,m,x)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))
def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))
def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 
def fOverField(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td


def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data



def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo
            

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    data=data+[sd]
print Set(data)

{1, 2, 3, 4}


In [11]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    order=prime^sd
    td=totalDegreeOverField(poly,prime,sd)
    lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
    logs=[]
    digits=[]
    multiplicities=[]
    for j in [0..len(lm)-1]:
        log=lm[j][0]
        multiplicity=lm[j][1]
        digit=log.digits(prime)
        digit= padListOnRightToLengthK(digit,sd)
        logs=logs+[log]
        multiplicities=multiplicities+[multiplicity]
        digits=digits+[digit]
    print
    print (index,sd,td,len(logs),multiplicities)
    print
    print logs;print;print digits
    print "--------------------------------------------------------------------------------------"

1

(-1, 1, 0, 0, [])

[]

[]
--------------------------------------------------------------------------------------
1

(0, 1, 0, 0, [])

[]

[]
--------------------------------------------------------------------------------------
x^2

(1, 1, 2, 1, [2])

[-1]

[[-1]]
--------------------------------------------------------------------------------------
(x + 1)^2 * (x + 2)^2

(2, 1, 4, 2, [2, 2])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^2 * (x + 2)^2

(3, 1, 6, 3, [2, 2, 2])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^2 * (x + 2)^2

(4, 1, 6, 3, [2, 2, 2])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + 1)^4 * (x + 2)^4

(5, 1, 8, 2, [4, 4])

[1, 2]

[[1], [2]]
------------------------------------------------------------------------------------

x^2 * (x + 1)^32 * (x + 2)^32

(48, 1, 66, 3, [2, 32, 32])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + t^2 + 2) * (x + t^2 + t) * (x + t^2 + 2*t) * (x + 2*t^2 + 1) * (x + 2*t^2 + t) * (x + 2*t^2 + 2*t) * x^2 * (x + 1)^30 * (x + 2)^30

(49, 3, 68, 9, [1, 1, 1, 1, 1, 1, 2, 30, 30])

[25, 23, 17, 12, 4, 10, -1, 13, 26]

[[1, 2, 2], [2, 1, 2], [2, 2, 1], [0, 1, 1], [1, 1, 0], [1, 0, 1], [-1, 0, 0], [1, 1, 1], [2, 2, 2]]
--------------------------------------------------------------------------------------
(2) * (x + t^2 + 2) * (x + t^2 + t) * (x + t^2 + 2*t) * (x + 2*t^2 + 1) * (x + 2*t^2 + t) * (x + 2*t^2 + 2*t) * (x + 1)^32 * (x + 2)^32

(50, 3, 70, 8, [1, 1, 1, 1, 1, 1, 32, 32])

[25, 23, 17, 12, 4, 10, 13, 26]

[[1, 2, 2], [2, 1, 2], [2, 2, 1], [0, 1, 1], [1, 1, 0], [1, 0, 1], [1, 1, 1], [2, 2, 2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^34 * 

(2) * x^2 * (x + 1)^64 * (x + 2)^64

(97, 1, 130, 3, [2, 64, 64])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * (x + 1)^66 * (x + 2)^66

(98, 1, 132, 2, [66, 66])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^66 * (x + 2)^66

(99, 1, 134, 3, [2, 66, 66])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^66 * (x + 2)^66

(100, 1, 134, 3, [2, 66, 66])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
x^6 * (x + 1)^66 * (x + 2)^66

(101, 1, 138, 3, [6, 66, 66])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^68 * (x + 2)^68

(102, 1, 138, 3, [2, 68, 68])

[-1, 1, 2]

[[-1], [1], [2]]
-----------

(2) * (x + t)^2 * (x + t + 1)^2 * (x + t + 2)^2 * (x + 2*t)^2 * (x + 2*t + 1)^2 * (x + 2*t + 2)^2 * x^8 * (x + 1)^88 * (x + 2)^88

(142, 2, 196, 9, [2, 2, 2, 2, 2, 2, 8, 88, 88])

[5, 6, 3, 1, 7, 2, -1, 4, 8]

[[2, 1], [0, 2], [0, 1], [1, 0], [1, 2], [2, 0], [-1, 0], [1, 1], [2, 2]]
--------------------------------------------------------------------------------------
(x + 1)^96 * (x + 2)^96

(143, 1, 192, 2, [96, 96])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^96 * (x + 2)^96

(144, 1, 194, 3, [2, 96, 96])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * x^8 * (x + 1)^94 * (x + 2)^94

(145, 1, 196, 3, [8, 94, 94])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + 1)^98 * (x + 2)^98

(146, 1, 196, 2, [98, 98])

[1, 2]

[[1], [2]]
------------------------------------

(x + 1)^124 * (x + 2)^124

(185, 1, 248, 2, [124, 124])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^124 * (x + 2)^124

(186, 1, 250, 3, [2, 124, 124])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * (x + t)^2 * (x + t + 1)^2 * (x + t + 2)^2 * (x + 2*t)^2 * (x + 2*t + 1)^2 * (x + 2*t + 2)^2 * x^8 * (x + 1)^118 * (x + 2)^118

(187, 2, 256, 9, [2, 2, 2, 2, 2, 2, 8, 118, 118])

[5, 6, 3, 1, 7, 2, -1, 4, 8]

[[2, 1], [0, 2], [0, 1], [1, 0], [1, 2], [2, 0], [-1, 0], [1, 1], [2, 2]]
--------------------------------------------------------------------------------------
(x + t) * (x + t + 1) * (x + t + 2) * (x + 2*t) * (x + 2*t + 1) * (x + 2*t + 2) * (x + 1)^124 * (x + 2)^124

(188, 2, 254, 8, [1, 1, 1, 1, 1, 1, 124, 124])

[5, 6, 3, 1, 7, 2, 4, 8]

[[2, 1], [0, 2], [0, 1], [1, 0], [1, 2], [2, 0], [1, 1], [2, 2]]
--------------------------------

In [12]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==1:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,len(logs),multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------------------------"

1

(-1, 1, 0, 0, [])

[]

[]
--------------------------------------------------------------------------------------
1

(0, 1, 0, 0, [])

[]

[]
--------------------------------------------------------------------------------------
x^2

(1, 1, 2, 1, [2])

[-1]

[[-1]]
--------------------------------------------------------------------------------------
(x + 1)^2 * (x + 2)^2

(2, 1, 4, 2, [2, 2])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^2 * (x + 2)^2

(3, 1, 6, 3, [2, 2, 2])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^2 * (x + 2)^2

(4, 1, 6, 3, [2, 2, 2])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + 1)^4 * (x + 2)^4

(5, 1, 8, 2, [4, 4])

[1, 2]

[[1], [2]]
------------------------------------------------------------------------------------

(2) * x^2 * (x + 1)^42 * (x + 2)^42

(64, 1, 86, 3, [2, 42, 42])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * (x + 1)^44 * (x + 2)^44

(65, 1, 88, 2, [44, 44])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^44 * (x + 2)^44

(66, 1, 90, 3, [2, 44, 44])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^44 * (x + 2)^44

(67, 1, 90, 3, [2, 44, 44])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + 1)^46 * (x + 2)^46

(68, 1, 92, 2, [46, 46])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^46 * (x + 2)^46

(69, 1, 94, 3, [2, 46, 46])

[-1, 1, 2]

[[-1], [1], [2]]
---------------------------------------------

(2) * x^2 * (x + 1)^80 * (x + 2)^80

(120, 1, 162, 3, [2, 80, 80])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^80 * (x + 2)^80

(121, 1, 162, 3, [2, 80, 80])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + 1)^82 * (x + 2)^82

(122, 1, 164, 2, [82, 82])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^82 * (x + 2)^82

(123, 1, 166, 3, [2, 82, 82])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^82 * (x + 2)^82

(124, 1, 166, 3, [2, 82, 82])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^84 * (x + 2)^84

(126, 1, 170, 3, [2, 84, 84])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------

x^2 * (x + 1)^116 * (x + 2)^116

(174, 1, 234, 3, [2, 116, 116])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
x^2 * (x + 1)^116 * (x + 2)^116

(175, 1, 234, 3, [2, 116, 116])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + 1)^118 * (x + 2)^118

(176, 1, 236, 2, [118, 118])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^118 * (x + 2)^118

(177, 1, 238, 3, [2, 118, 118])

[-1, 1, 2]

[[-1], [1], [2]]
--------------------------------------------------------------------------------------
(x + 1)^120 * (x + 2)^120

(179, 1, 240, 2, [120, 120])

[1, 2]

[[1], [2]]
--------------------------------------------------------------------------------------
(2) * x^2 * (x + 1)^120 * (x + 2)^120

(180, 1, 242, 3, [2, 120, 120])

[-1, 1, 2]

[[-1], [1], [2]]
---------------------

In [13]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==2:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        print
        print (index,sd,td,len(logs),multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------"

(2) * (x + t)^2 * (x + t + 1)^2 * (x + t + 2)^2 * (x + 2*t)^2 * (x + 2*t + 1)^2 * (x + 2*t + 2)^2 * x^8 * (x + 1)^16 * (x + 2)^16

(34, 2, 52, 9, [2, 2, 2, 2, 2, 2, 8, 16, 16])

[5, 6, 3, 1, 7, 2, -1, 4, 8]

[[2, 1], [0, 2], [0, 1], [1, 0], [1, 2], [2, 0], [-1, 0], [1, 1], [2, 2]]
--------------------------------------------------------------------
(x + t) * (x + t + 1) * (x + t + 2) * (x + 2*t) * (x + 2*t + 1) * (x + 2*t + 2) * (x + 1)^52 * (x + 2)^52

(80, 2, 110, 8, [1, 1, 1, 1, 1, 1, 52, 52])

[5, 6, 3, 1, 7, 2, 4, 8]

[[2, 1], [0, 2], [0, 1], [1, 0], [1, 2], [2, 0], [1, 1], [2, 2]]
--------------------------------------------------------------------
(x + t) * (x + t + 1) * (x + t + 2) * (x + 2*t) * (x + 2*t + 1) * (x + 2*t + 2) * (x + 1)^54 * (x + 2)^54

(83, 2, 114, 8, [1, 1, 1, 1, 1, 1, 54, 54])

[5, 6, 3, 1, 7, 2, 4, 8]

[[2, 1], [0, 2], [0, 1], [1, 0], [1, 2], [2, 0], [1, 1], [2, 2]]
--------------------------------------------------------------------
(2) * (x + t)^2 * (x + t 

In [37]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
loglenths=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==3:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        ll=len(logs)
        print
        print (index,sd,td,ll,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------"
        loglenths=loglenths+[ll]
print loglenths

(2) * (x + t^2 + 2) * (x + t^2 + t) * (x + t^2 + 2*t) * (x + 2*t^2 + 1) * (x + 2*t^2 + t) * (x + 2*t^2 + 2*t) * (x + 1)^4 * (x + 2)^4

(8, 3, 14, 8, [1, 1, 1, 1, 1, 1, 4, 4])

[25, 23, 17, 12, 4, 10, 13, 26]

[[1, 2, 2], [2, 1, 2], [2, 2, 1], [0, 1, 1], [1, 1, 0], [1, 0, 1], [1, 1, 1], [2, 2, 2]]
--------------------------------------------------------------------
(2) * (x + t^2 + 2) * (x + t^2 + t) * (x + t^2 + 2*t) * (x + 2*t^2 + 1) * (x + 2*t^2 + t) * (x + 2*t^2 + 2*t) * (x + 1)^6 * (x + 2)^6

(11, 3, 18, 8, [1, 1, 1, 1, 1, 1, 6, 6])

[25, 23, 17, 12, 4, 10, 13, 26]

[[1, 2, 2], [2, 1, 2], [2, 2, 1], [0, 1, 1], [1, 1, 0], [1, 0, 1], [1, 1, 1], [2, 2, 2]]
--------------------------------------------------------------------
(x + t^2 + 2) * (x + t^2 + t) * (x + t^2 + 2*t) * (x + 2*t^2 + 1) * (x + 2*t^2 + t) * (x + 2*t^2 + 2*t) * (x + 1)^14 * (x + 2)^14

(23, 3, 34, 8, [1, 1, 1, 1, 1, 1, 14, 14])

[25, 23, 17, 12, 4, 10, 13, 26]

[[1, 2, 2], [2, 1, 2], [2, 2, 1], [0, 1, 1], [1, 1, 0], [

In [38]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
loglenths=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==4:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverFieldTest(poly,prime,sd)
        logs=[]
        digits=[]
        multiplicities=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            multiplicity=lm[j][1]
            digit=log.digits(prime)
            digit= padListOnRightToLengthK(digit,sd)
            logs=logs+[log]
            multiplicities=multiplicities+[multiplicity]
            digits=digits+[digit]
        ll=len(logs)
        print
        print (index,sd,td,ll,multiplicities)
        print
        print logs;print;print digits
        print "--------------------------------------------------------------------"
        loglenths=loglenths+[ll]
print loglenths

(x + t^2 + t + 1) * (x + t^2 + 2*t) * (x + 2*t^2 + t) * (x + 2*t^2 + 2*t + 2) * (x + t^3 + t + 1) * (x + t^3 + 2*t) * (x + t^3 + 2*t^2 + t + 2) * (x + t^3 + 2*t^2 + 2*t + 1) * (x + 2*t^3 + t) * (x + 2*t^3 + 2*t + 2) * (x + 2*t^3 + t^2 + t + 2) * (x + 2*t^3 + t^2 + 2*t + 1) * (x + 1)^8 * (x + 2)^8

(17, 4, 28, 14, [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 8])

[34, 38, 78, 74, 45, 66, 15, 62, 26, 5, 22, 55, 40, 80]

[[1, 2, 0, 1], [2, 0, 1, 1], [0, 2, 2, 2], [2, 0, 2, 2], [0, 0, 2, 1], [0, 1, 1, 2], [0, 2, 1, 0], [2, 2, 0, 2], [2, 2, 2, 0], [2, 1, 0, 0], [1, 1, 2, 0], [1, 0, 0, 2], [1, 1, 1, 1], [2, 2, 2, 2]]
--------------------------------------------------------------------
[14]


In [29]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
twodata=[]
threedata=[]
otherdata=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==1:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        ll=len(logs)
        if ll==2:twodata=twodata+[index]
        if ll==3:threedata=threedata+[index]
        if ll>3:otherdata=otherdata+[index]
print twodata
print
print threedata
print
print otherdata
            

[2, 5, 14, 20, 26, 29, 32, 35, 41, 44, 47, 53, 56, 59, 62, 65, 68, 71, 74, 77, 89, 92, 98, 107, 110, 113, 119, 122, 128, 131, 134, 137, 140, 143, 146, 149, 152, 155, 158, 161, 164, 167, 170, 176, 179, 182, 185, 197]

[3, 4, 6, 7, 9, 10, 12, 13, 15, 16, 18, 19, 21, 22, 24, 25, 27, 28, 30, 31, 33, 36, 37, 39, 40, 42, 43, 45, 46, 48, 51, 52, 54, 55, 57, 58, 60, 61, 63, 64, 66, 67, 69, 70, 72, 73, 75, 76, 78, 79, 81, 82, 84, 86, 87, 88, 90, 91, 93, 94, 95, 96, 97, 99, 100, 101, 102, 103, 105, 108, 109, 111, 112, 114, 115, 117, 118, 120, 121, 123, 124, 126, 127, 129, 130, 132, 133, 135, 138, 139, 141, 144, 145, 147, 148, 150, 151, 153, 154, 156, 157, 159, 162, 163, 165, 166, 168, 171, 172, 173, 174, 175, 177, 180, 181, 183, 184, 186, 189, 190, 191, 192, 193, 195, 198, 199]

[]


In [30]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
twodata=[]
threedata=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==1:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        ll=len(logs)
        if ll==2:twodata=twodata+[mod(index,3)]
        if ll==3:threedata=threedata+[mod(index,3)]
print twodata
print
print threedata
print
print otherdata
            

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 1, 0, 1, 0, 1, 2, 0, 1, 0, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 2, 0, 1, 0, 0, 1]

[]


In [35]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data8=[]
data9=[]
other=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==2:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        ll=len(logs)
        if ll==8:data8=data8+[index]
        if ll==9:data9=data9+[index]
        if ll<8:other=other+[index]
        if ll>9:other=other+[index]
print data8
print
print data9
print
print other
            

[80, 83, 116, 125, 188, 194]

[34, 85, 136, 142, 160, 178, 187]

[]


In [36]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data8=[]
data9=[]
other=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==2:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        ll=len(logs)
        if ll==8:data8=data8+[mod(index,3)]
        if ll==9:data9=data9+[mod(index,3)]
print data8
print
print data9
print
print other

[2, 2, 2, 2, 2, 2]

[1, 1, 1, 1, 1, 1, 1]

[]


In [ ]:
#[8, 8, 8, 8, 9, 8, 8, 21, 9, 9]

In [42]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data8=[]
data9=[]
data21=[]
other=[]
for k in [0..len(s)-1]:
    index=s[k][0]
    zerodata=[]
    poly = s[k][1]
    poly=polynomialnumerator(D(poly))
    sd=leastSplittingDegree(poly,prime,bound)
    if sd==3:
        td=totalDegreeOverField(poly,prime,sd)
        lm=logsAndMultiplicitiesOverField(poly,prime,sd)
        logs=[]
        for j in [0..len(lm)-1]:
            log=lm[j][0]
            logs=logs+[log]
        ll=len(logs)
        if ll==8:data8=data8+[mod(index,3)]
        if ll==9:data9=data9+[mod(index,27)]
        if ll==21:data21=data21+[mod(index,27)]
print data8
print
print data9
print
print data21

[2, 2, 2, 2, 2, 2]

[22, 7, 7]

[25]
